In [1]:
using Pkg
Pkg.status()

      Status `~/Mathematics/Conferences/Presentations/SumOfSquaresSlides/2022_ICM_Computational_Math/Project.toml`
  [7c1d4256] DynamicPolynomials v0.4.5
  [7073ff75] IJulia v1.23.3
  [4076af6c] JuMP v1.1.1
  [8bc5a954] PermutationGroups v0.3.2
  [c946c3f1] SCS v1.1.2
  [4b9e565b] SumOfSquares v0.6.2
  [24249f21] SymPy v1.1.6
  [858aa9a9] SymbolicWedderburn v0.3.0


In [2]:
using LinearAlgebra
using SumOfSquares
using SCS
using DynamicPolynomials

In [3]:
const N = 4
@polyvar x[1:N]

(PolyVar{true}[x₁, x₂, x₃, x₄],)

In [4]:
f = sum(x) +
    sum(x .^ 2) +
    (sum((x .+ 1) .^ 2 .* (x .+ x') .^ 2)*sum(x)^2)
(f,)

(7x₁⁶ + 16x₁⁵x₂ + 16x₁⁵x₃ + 16x₁⁵x₄ + 13x₁⁴x₂² + 22x₁⁴x₂x₃ + 22x₁⁴x₂x₄ + 13x₁⁴x₃² + 22x₁⁴x₃x₄ + 13x₁⁴x₄² + 8x₁³x₂³ + 10x₁³x₂²x₃ + 10x₁³x₂²x₄ + 10x₁³x₂x₃² + 12x₁³x₂x₃x₄ + 10x₁³x₂x₄² + 8x₁³x₃³ + 10x₁³x₃²x₄ + 10x₁³x₃x₄² + 8x₁³x₄³ + 13x₁²x₂⁴ + 10x₁²x₂³x₃ + 10x₁²x₂³x₄ + 6x₁²x₂²x₃² + 4x₁²x₂²x₃x₄ + 6x₁²x₂²x₄² + 10x₁²x₂x₃³ + 4x₁²x₂x₃²x₄ + 4x₁²x₂x₃x₄² + 10x₁²x₂x₄³ + 13x₁²x₃⁴ + 10x₁²x₃³x₄ + 6x₁²x₃²x₄² + 10x₁²x₃x₄³ + 13x₁²x₄⁴ + 16x₁x₂⁵ + 22x₁x₂⁴x₃ + 22x₁x₂⁴x₄ + 10x₁x₂³x₃² + 12x₁x₂³x₃x₄ + 10x₁x₂³x₄² + 10x₁x₂²x₃³ + 4x₁x₂²x₃²x₄ + 4x₁x₂²x₃x₄² + 10x₁x₂²x₄³ + 22x₁x₂x₃⁴ + 12x₁x₂x₃³x₄ + 4x₁x₂x₃²x₄² + 12x₁x₂x₃x₄³ + 22x₁x₂x₄⁴ + 16x₁x₃⁵ + 22x₁x₃⁴x₄ + 10x₁x₃³x₄² + 10x₁x₃²x₄³ + 22x₁x₃x₄⁴ + 16x₁x₄⁵ + 7x₂⁶ + 16x₂⁵x₃ + 16x₂⁵x₄ + 13x₂⁴x₃² + 22x₂⁴x₃x₄ + 13x₂⁴x₄² + 8x₂³x₃³ + 10x₂³x₃²x₄ + 10x₂³x₃x₄² + 8x₂³x₄³ + 13x₂²x₃⁴ + 10x₂²x₃³x₄ + 6x₂²x₃²x₄² + 10x₂²x₃x₄³ + 13x₂²x₄⁴ + 16x₂x₃⁵ + 22x₂x₃⁴x₄ + 10x₂x₃³x₄² + 10x₂x₃²x₄³ + 22x₂x₃x₄⁴ + 16x₂x₄⁵ + 7x₃⁶ + 16x₃⁵x₄ + 13x₃⁴x₄² + 8x₃³x₄³ + 13x₃²x₄⁴ + 16x₃x₄⁵ + 7x₄⁶ + 14x₁⁵ + 34x

In [5]:
scs_opt(;eps=1e-5, verbose=true) = optimizer_with_attributes(
    SCS.Optimizer, "eps_abs"=>eps, "eps_rel"=>eps, "verbose"=>verbose, "acceleration_lookback"=>0)

scs_opt (generic function with 1 method)

In [6]:
no_symmetry = let f=f, m = SOSModel()
    @variable m t
    @objective m Max t
    @constraint m f-t in SOSCone()
    m
end

A JuMP Model
Maximization problem with:
Variable: 1
Objective function type: VariableRef
`Vector{AffExpr}`-in-`SumOfSquares.SOSPolynomialSet{FullSpace, Monomial{true}, MonomialVector{true}, SumOfSquares.Certificate.Newton{SOSCone, MonomialBasis, Tuple{}}}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: t

In [7]:
set_optimizer(no_symmetry, scs_opt(verbose=true))
optimize!(no_symmetry);
@info termination_status(no_symmetry)
objective_value(no_symmetry)

------------------------------------------------------------------
	       SCS v3.2.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 631, constraints m: 840
cones: 	  z: primal zero / dual free vars: 210
	  s: psd vars: 630, ssize: 1
settings: eps_abs: 1.0e-05, eps_rel: 1.0e-05, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
lin-sys:  sparse-direct
	  nnz(A): 1261, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 5.20e+01  3.00e+00  3.74e+03  1.85e+03  1.00e-01  1.73e-03 
   250| 2.22e-03  4.34e-05  2.09e-02  2.67e-01  1.00e-01  6.48e-02 
   500| 6.36e-04  2.17e-05  2.31e-02  2.66e-01  1.00e-01  1.19e-01 
   750| 4.95e-04  1.59

┌ Info: OPTIMAL
└ @ Main In[7]:3


-0.27707976088580943

# Defining group action for the first time

In [8]:
using GroupsCore
using PermutationGroups
using SymbolicWedderburn

In [9]:
G = PermGroup([perm"(1,2)", Perm([2:N; 1])])

Permutation group on 2 generators generated by
 (1,2)
 (1,2,3,4)

In [10]:
struct VariablePermutation{V} <: SymbolicWedderburn.ByPermutations
    vars::V
end

function SymbolicWedderburn.action(
    a::VariablePermutation,
    el::GroupElement,
    poly::DynamicPolynomials.AbstractPolynomial,
)
    return sum(SymbolicWedderburn.action(a, el, term) for term in DynamicPolynomials.terms(poly))
end

function SymbolicWedderburn.action(
    a::VariablePermutation,
    el::GroupElement,
    term::DynamicPolynomials.AbstractTerm,
)
    return DynamicPolynomials.coefficient(term) *
           SymbolicWedderburn.action(a, el, DynamicPolynomials.monomial(term))
end

function SymbolicWedderburn.action(
    a::VariablePermutation,
    g::PermutationGroups.AbstractPerm,
    m::DynamicPolynomials.Monomial,
)
    return m(a.vars => a.vars^g)
end

In [11]:
ac = VariablePermutation(x)

VariablePermutation{Vector{PolyVar{true}}}(PolyVar{true}[x₁, x₂, x₃, x₄])

In [12]:
SymbolicWedderburn.action(ac, perm"(1,2,3,4)", 3*x[1]*x[2] + 2*x[1]*x[3])

3x₂x₃ + 2x₂x₄

In [13]:
for g in G
    @assert SymbolicWedderburn.action(ac, g, f) == f
end

## Reducing the number of constraints

In [29]:
orbit_reduction = let poly = f, sos_model = JuMP.Model()
    JuMP.@variable sos_model t
    JuMP.@objective sos_model Max t
    n = let vars = variables(poly), mdeg = DynamicPolynomials.maxdegree(poly)
        length(DynamicPolynomials.monomials(vars, 0:mdeg÷2))
    end
    P = JuMP.@variable sos_model P[1:n, 1:n] Symmetric
    JuMP.@constraint sos_model P in PSDCone()
    sos_model
end

A JuMP Model
Maximization problem with:
Variables: 631
Objective function type: VariableRef
`Vector{VariableRef}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: P, t

In [30]:
function orbit_decomposition(
        action::SymbolicWedderburn.Action,
        G::Group,
        basis::SymbolicWedderburn.StarAlgebras.AbstractBasis,
)
    b = SymbolicWedderburn.StarAlgebras.Basis{UInt32}
    tbl = SymbolicWedderburn.Characters.CharacterTable(Rational{Int}, G)
    return SymbolicWedderburn.invariant_vectors(tbl, action, basis)
end

orbit_decomposition (generic function with 1 method)

In [32]:
max_deg = DynamicPolynomials.maxdegree(f)
vars = DynamicPolynomials.variables(f)
basis_psd = DynamicPolynomials.monomials(vars, 0:max_deg÷2)
basis_constraints = SymbolicWedderburn.StarAlgebras.Basis{UInt32}(DynamicPolynomials.monomials(vars, 0:max_deg))
# @info "constraints are written in the following basis" basis=basis_constraints

orbit_dec = orbit_decomposition(ac, G, basis_constraints)

# [v'*basis_constraints for v in orbit_dec][1:2]

27-element Vector{SparseArrays.SparseVector{Rational{Int64}}}:
   [1  ]  =  1//4
  [57 ]  =  1//4
  [78 ]  =  1//4
  [84 ]  =  1//4
   [2  ]  =  1//12
  [3  ]  =  1//12
  [4  ]  =  1//12
  [36 ]  =  1//12
  [51 ]  =  1//12
  [56 ]  =  1//12
  [58 ]  =  1//12
  [59 ]  =  1//12
  [72 ]  =  1//12
  [77 ]  =  1//12
  [79 ]  =  1//12
  [83 ]  =  1//12
   [5  ]  =  1//12
  [8  ]  =  1//12
  [10 ]  =  1//12
  [21 ]  =  1//12
  [31 ]  =  1//12
  [35 ]  =  1//12
  [60 ]  =  1//12
  [62 ]  =  1//12
  [67 ]  =  1//12
  [71 ]  =  1//12
  [80 ]  =  1//12
  [82 ]  =  1//12
   [6  ]  =  1//12
  [7  ]  =  1//12
  [9  ]  =  1//12
  [37 ]  =  1//12
  [38 ]  =  1//12
  [46 ]  =  1//12
  [50 ]  =  1//12
  [52 ]  =  1//12
  [55 ]  =  1//12
  [61 ]  =  1//12
  [73 ]  =  1//12
  [76 ]  =  1//12
   [11 ]  =  1//6
  [17 ]  =  1//6
  [20 ]  =  1//6
  [63 ]  =  1//6
  [66 ]  =  1//6
  [81 ]  =  1//6
   [12 ]  =  1//24
  [13 ]  =  1//24
  [14 ]  =  1//24
  [16 ]  =  1//24
  [18 ]  =  1//24
  [19 ]  =  1//24
  [22

In [33]:
let sos_problem = orbit_reduction, poly = f, P = orbit_reduction[:P]

    A, C = let m = basis_psd*basis_psd', t = sos_problem[:t]
        A = [m .== t for t in basis_constraints]
        C = DynamicPolynomials.coefficients(poly - t, basis_constraints)
        A, C
    end
    
    for V in orbit_dec
        c = dot(V, C)
        # average A with weights given by V:
        A_orb = sum(v.*A[t] for (t,v) in pairs(V))
        JuMP.@constraint sos_problem dot(A_orb, P) == c
    end

    sos_problem
end

A JuMP Model
Maximization problem with:
Variables: 631
Objective function type: VariableRef
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 27 constraints
`Vector{VariableRef}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: P, t

In [34]:
set_optimizer(orbit_reduction, scs_opt(verbose=true))
optimize!(orbit_reduction)

------------------------------------------------------------------
	       SCS v3.2.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 631, constraints m: 657
cones: 	  z: primal zero / dual free vars: 27
	  s: psd vars: 630, ssize: 1
settings: eps_abs: 1.0e-05, eps_rel: 1.0e-05, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
lin-sys:  sparse-direct
	  nnz(A): 1261, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 5.20e+01  2.33e+00  2.92e+03  1.43e+03  1.00e-01  7.99e-04 
   250| 2.32e-03  2.53e-05  2.45e-02  2.65e-01  1.00e-01  8.85e-02 
   500| 6.48e-04  9.62e-06  2.62e-02  2.64e-01  1.00e-01  1.39e-01 
   750| 4.96e-04  5.55e

# Blocks, Blocks, Blocks

In [35]:
basis_psd

35-element MonomialVector{true}:
 x₁³
 x₁²x₂
 x₁²x₃
 x₁²x₄
 x₁x₂²
 x₁x₂x₃
 x₁x₂x₄
 x₁x₃²
 x₁x₃x₄
 x₁x₄²
 x₂³
 x₂²x₃
 x₂²x₄
 ⋮
 x₁x₄
 x₂²
 x₂x₃
 x₂x₄
 x₃²
 x₃x₄
 x₄²
 x₁
 x₂
 x₃
 x₄
 1

In [36]:
wdec_ssimple = WedderburnDecomposition(
    Float64,
    G,
    ac,
    basis_constraints,
    basis_psd,
    semisimple = true,
)

Wedderburn Decomposition into 27 orbits and 4 summands of sizes
[4, 21, 7, 3]

In [37]:
round.(direct_summands(wdec_ssimple)[3]*basis_psd, digits=3)

7-element Vector{Polynomial{true, Float64}}:
 1.0
 -0.5x₁³ - 0.5x₂³ - 0.5x₃³ - 0.5x₄³
 -0.5x₁x₂x₃ - 0.5x₁x₂x₄ - 0.5x₁x₃x₄ - 0.5x₂x₃x₄
 -0.5x₁² - 0.5x₂² - 0.5x₃² - 0.5x₄²
 -0.5x₁ - 0.5x₂ - 0.5x₃ - 0.5x₄
 -0.408x₁x₂ - 0.408x₁x₃ - 0.408x₁x₄ - 0.408x₂x₃ - 0.408x₂x₄ - 0.408x₃x₄
 -0.289x₁²x₂ - 0.289x₁²x₃ - 0.289x₁²x₄ - 0.289x₁x₂² - 0.289x₁x₃² - 0.289x₁x₄² - 0.289x₂²x₃ - 0.289x₂²x₄ - 0.289x₂x₃² - 0.289x₂x₄² - 0.289x₃²x₄ - 0.289x₃x₄²

In [38]:
semisimple_model, psds = let sos_model = JuMP.Model(), wedderburn = wdec_ssimple
    JuMP.@variable sos_model t
    JuMP.@objective sos_model Max t
    psds = map(SymbolicWedderburn.direct_summands(wedderburn)) do ds
        dim = size(ds, 1)
        P = JuMP.@variable sos_model [1:dim, 1:dim] Symmetric
        JuMP.@constraint sos_model P in PSDCone()
        P
    end
    sos_model, psds
end

(A JuMP Model
Maximization problem with:
Variables: 276
Objective function type: VariableRef
`Vector{VariableRef}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 4 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: t, Symmetric{VariableRef, Matrix{VariableRef}}[[_[2] _[3] _[5] _[8]; _[3] _[4] _[6] _[9]; _[5] _[6] _[7] _[10]; _[8] _[9] _[10] _[11]], [_[12] _[13] … _[202] _[222]; _[13] _[14] … _[203] _[223]; … ; _[202] _[203] … _[221] _[241]; _[222] _[223] … _[241] _[242]], [_[243] _[244] … _[258] _[264]; _[244] _[245] … _[259] _[265]; … ; _[258] _[259] … _[263] _[269]; _[264] _[265] … _[269] _[270]], [_[271] _[272] _[274]; _[272] _[273] _[275]; _[274] _[275] _[276]]])

In [39]:
let sos_model = semisimple_model, wedderburn = wdec_ssimple, poly = f, P = psds
    
    A, C = let m = basis_psd*basis_psd', t = sos_model[:t]
        A = [m .== t for t in basis_constraints]
        C = DynamicPolynomials.coefficients(poly - t, basis_constraints)
        A, C
    end
    
    for V in invariant_vectors(wedderburn)
        c = dot(V, C)
        # average A with weights given by V:
        A_orb = sum(v.*A[t] for (t,v) in pairs(V))
        
        dA = SymbolicWedderburn.diagonalize(A_orb, wedderburn)
        JuMP.@constraint sos_model sum(
            dot(dA[π], P[π]) for π in 1:length(P)
        ) == c
    end
    sos_model
end

A JuMP Model
Maximization problem with:
Variables: 276
Objective function type: VariableRef
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 27 constraints
`Vector{VariableRef}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 4 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: t

In [40]:
set_optimizer(semisimple_model, scs_opt(verbose=true))
optimize!(semisimple_model)

------------------------------------------------------------------
	       SCS v3.2.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 276, constraints m: 302
cones: 	  z: primal zero / dual free vars: 27
	  s: psd vars: 275, ssize: 4
settings: eps_abs: 1.0e-05, eps_rel: 1.0e-05, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
lin-sys:  sparse-direct
	  nnz(A): 861, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 5.19e+01  9.65e+00  2.54e+03  1.24e+03  1.00e-01  4.97e-04 
   250| 1.35e-02  2.94e-05  3.15e-02  2.62e-01  2.77e-02  4.59e-02 
   500| 2.79e-03  4.58e-06  3.05e-02  2.62e-01  2.77e-02  8.29e-02 
   750| 2.68e-03  2.98e-

In [41]:
wdec_simple = WedderburnDecomposition(
    Float64,
    G,
    ac,
    basis_constraints,
    basis_psd,
    semisimple = false,
)

Wedderburn Decomposition into 27 orbits and 4 simple summands of sizes
[2, 7, 7, 1]

In [42]:
simple_model, psds = let sos_model = JuMP.Model(), wedderburn = wdec_simple
    JuMP.@variable sos_model t
    JuMP.@objective sos_model Max t
    psds = map(SymbolicWedderburn.direct_summands(wedderburn)) do ds
        dim = size(ds, 1)
        P = JuMP.@variable sos_model [1:dim, 1:dim] Symmetric
        JuMP.@constraint sos_model P in PSDCone()
        P
    end
    sos_model, psds
end

(A JuMP Model
Maximization problem with:
Variables: 61
Objective function type: VariableRef
`Vector{VariableRef}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 4 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: t, Symmetric{VariableRef, Matrix{VariableRef}}[[_[2] _[3]; _[3] _[4]], [_[5] _[6] … _[20] _[26]; _[6] _[7] … _[21] _[27]; … ; _[20] _[21] … _[25] _[31]; _[26] _[27] … _[31] _[32]], [_[33] _[34] … _[48] _[54]; _[34] _[35] … _[49] _[55]; … ; _[48] _[49] … _[53] _[59]; _[54] _[55] … _[59] _[60]], [_[61];;]])

In [43]:
let sos_model = simple_model, wedderburn = wdec_simple, poly = f, P = psds
    
    A, C = let m = basis_psd*basis_psd', t = sos_model[:t]
        A = [m .== t for t in basis_constraints]
        C = DynamicPolynomials.coefficients(poly - t, basis_constraints)
        A, C
    end
    
    for V in invariant_vectors(wedderburn)
        c = dot(V, C)
        # average A with weights given by V:
        A_orb = sum(v.*A[t] for (t,v) in pairs(V))
        
        dA = SymbolicWedderburn.diagonalize(A_orb, wedderburn)
        JuMP.@constraint sos_model sum(
            dot(dA[π], P[π]) for π in 1:length(P)
        ) == c
    end
    sos_model
end

A JuMP Model
Maximization problem with:
Variables: 61
Objective function type: VariableRef
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 27 constraints
`Vector{VariableRef}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 4 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: t

In [44]:
set_optimizer(semisimple_model, scs_opt(verbose=true))
optimize!(semisimple_model)

------------------------------------------------------------------
	       SCS v3.2.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 276, constraints m: 302
cones: 	  z: primal zero / dual free vars: 27
	  s: psd vars: 275, ssize: 4
settings: eps_abs: 1.0e-05, eps_rel: 1.0e-05, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
lin-sys:  sparse-direct
	  nnz(A): 861, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 5.19e+01  9.65e+00  2.54e+03  1.24e+03  1.00e-01  5.79e-04 
   250| 1.35e-02  2.94e-05  3.15e-02  2.62e-01  2.77e-02  3.54e-02 
   500| 2.79e-03  4.58e-06  3.05e-02  2.62e-01  2.77e-02  6.85e-02 
   750| 2.68e-03  2.98e-